# Tester notebook for the LDA dimensionality reduction method

In [52]:
# Imports

#Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Feature extraction (PCA)
# from sklearn.base import BaseEstimator, ClassifierMixin
# from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [53]:
# Load data
trainDataNP = np.load("fashion_train.npy")
testDataNP = np.load("fashion_test.npy")

# Split data into X and y arrays
X_train = trainDataNP[:, :-1]
y_train = trainDataNP[:, -1]
X_test = testDataNP[:, :-1]
y_test = testDataNP[:, -1]

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [54]:
# Create LDA class

class LDA:
    def __init__(self):
        pass

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.class_labels = np.unique(y_train)
        self.coeffs, self.constants = self.compute_coeffs_and_constants()
        return self

    def compute_class_stats(self, class_label):
        '''Compute class specific statistics.'''
        class_data = self.X_train[self.y_train == class_label]
        class_sample_mean = np.mean(class_data, axis = 0)
        class_sample_var = np.var(class_data, axis = 0)
        class_sample_prior = np.sum(self.y_train == class_label) / len(self.y_train)
        return class_sample_mean, class_sample_var, class_sample_prior

    def compute_coeffs_and_constants(self):
        '''Compute coefficients and constants for each class.'''
        coeffs = {} # Initialize empty dictionary for coefficients
        constants = {} # Initialize empty dictionary for constants
        epsilon = 1e-12 # Small constant to avoid division by zero

        # Iterate over each class
        for class_label in self.class_labels:

            # Get statistics for class
            sample_mean, sample_var, sample_prior = self.compute_class_stats(class_label)
            
            # Compute coefficient
            alpha = sample_mean / (sample_var + epsilon)

            # Compute constant
            beta = -(np.sum(sample_mean**2) / (2 * np.sum(sample_var))) + np.log(sample_prior)

            # Record coefficient and constant in dictionaries
            coeffs[class_label] = alpha
            constants[class_label] = beta

        return coeffs, constants

    def discriminant(self, X, alpha, beta):
        '''Compute the discriminant value for an input value or array X.'''
        return np.dot(X, alpha) + beta

    def predict(self, X):
        '''Predict class for an input value or array X.'''
        # check_is_fitted(self)
        # X = check_array(X)

        # Compute the discriminant values for each label
        discriminants = np.array([self.discriminant(X, self.coeffs[class_label], self.constants[class_label]) for class_label in self.class_labels])
        
        # Predict classes and add labels
        predicted_labels = np.argmax(discriminants, axis = 0)
        predicted_classes = np.array(self.class_labels)[predicted_labels]
        
        return predicted_classes

    def transform(self, X, n_components):
        '''Project data onto the linear discriminant space.'''
        # check_is_fitted(self)
        # X = check_array(X)

        # Initialize empty array to store projected data
        projected_data = np.zeros((len(X), len(self.class_labels)))

        # Iterate over each class
        for idx, class_label in enumerate(self.class_labels):
            # Project data using the pre-computed coefficients and constants
            # projected_data[:, idx] = np.dot(X, self.coeffs[class_label]) + self.constants[class_label]

            # Compute the discriminant values
            discriminants = self.discriminant(X, self.coeffs[class_label], self.constants[class_label])

            # Project data using the discriminant value
            projected_data[:, idx] = discriminant

        return projected_data[:, :n_components]

    def score(self, X, y):
        '''Compute accuracy for the given input data and labels.'''
        # check_is_fitted(self)
        # X = check_array(X)
        # y = check_array(y)
        y_pred = self.predict(X)
        accuracy = np.mean(y_pred == y)
        return accuracy

In [55]:
# Check accuracy

# Initialize LDA model
lda_model = LDA()
lda_model.fit(X_train_scaled, y_train)

accuracy = lda_model.score(X_test_scaled, y_test)
print(accuracy)

0.205


Very low..

In [105]:
# Create new Multiple Discriminant Analysis (MDA) class which maybe works better
# Code is adapted using 
# Alpaydin, E. (2014). Introduction to Machine Learning. The MIT Press. Section 6.8
# and
# https://www.csd.uwo.ca/~oveksler/Courses/CS434a_541a/Lecture8.pdf?fbclid=IwAR2txae5XGTwi8Gjo2y9SbwguiJ0SUbLvCBzY0rYq-HOxhfr2MAjPo1GG3A

class MDA:
    def __init__(self, k = 2):
        self.k = k # Amount of discriminant variables
        
    def fit_transform(self, X, y):

        class_labels = np.unique(y)
        m = X.shape[1] # Amount of features

        # Compute total mean of all samples
        mu = np.mean(X, axis = 0)

        # Initiate within- and between-class scatter matrices as empty
        S_W = np.zeros((m, m))
        S_B = np.zeros((m, m))

        # Compute within- and between-class scatter matrices by iterating over each class i
        for label in class_labels:

            X_i = X[y == label] # Matrix of data points for class i
            mu_i = np.mean(X_i, axis = 0) # Sample mean of class i

            # Update within-class scatter matrix using class observations and mean
            # Use transpose first in matrix multiplation to maintain shape of S_W as (m, m)
            S_W += np.dot((X_i - mu_i).T, X_i - mu_i)

            # Update between-class scatter matrix using amount of samples, as well as class and total means
            # Use transpose first in matrix multiplation to maintain shape of S_B as (m, m)
            n_i = X_i.shape[0] # Amount of samples in class i
            S_B += n_i * np.dot((mu_i - mu).T, mu_i - mu)

        # Compute the inverse of the within-class scatter matrix
        S_W_inv = np.linalg.inv(S_W)

        # Solve the generalized eigenvalue problem
        eigenvalues, eigenvectors = np.linalg.eig(np.dot(S_W_inv, S_B))
        
        # Sort the eigenvalues and eigenvectors in descending order
        sorted_indices = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[sorted_indices]
        eigenvectors = eigenvectors[:, sorted_indices]

        # Select the top k eigenvectors
        selected_eigenvectors = eigenvectors[:, :self.k]

        # Make sure numbers in eigenvectors are not complex
        selected_eigenvectors = np.real(selected_eigenvectors)

        # Normalize eigenvectors
        normalized_eigenvectors = selected_eigenvectors / np.linalg.norm(selected_eigenvectors, axis = 0)

        # Project the data onto the normalized discriminant subspace
        X_projected = np.dot(X, normalized_eigenvectors)

        return X_projected

In [106]:
mda = MDA()
X_projected = mda.fit_transform(X_train_scaled, y_train)

In [107]:
X_projected

array([[-0.00624132, -0.12107445],
       [ 0.08716425,  0.09764791],
       [ 0.04659335, -0.05428298],
       ...,
       [ 0.02451672,  0.15719014],
       [ 0.10742126, -0.05401376],
       [ 0.0745882 ,  0.03366667]])

In [108]:
sklearn_MDA = LinearDiscriminantAnalysis(n_components = 2)
sklearn_MDA.fit(X_train, y_train)
sklearn_MDA.transform(X_train)

array([[-2.56669806, -0.55699892],
       [ 5.20026605,  1.0529189 ],
       [-1.9471824 , -0.48922497],
       ...,
       [-0.22086478, -3.5033158 ],
       [-1.43054366, -0.89234007],
       [ 6.27484131,  0.72650406]])